# 目的
1. 住所から、その緯度経度を取得
2. その緯度経度が中心点となるxx km四方の区画のメッシュ別統計量の一覧を取得
3. folium,osmで地図上に可視化

In [10]:
import numpy as np
from pandas import DataFrame
import pandas as pd
import requests
import folium
import geocoder

In [12]:
def callGeocode(address):
    g = geocoder.google(address)
    return g.latlng

In [39]:
jobsmesh_statics_list = pd.Series(['世界メッシュコード', '最低標高 (m)', '平均標高 (m)', '最高標高 (m)', '最高標高と最低標高との差 (m)',
                          '最小傾斜角 (度)', '平均傾斜角 (度)', '最大傾斜角 (度)',
                                   '人口 (人)', '人口密度 (人/km2)', '世帯数 (世帯)', '男性 (人)', '女性 (人)', '事業所数', '労働者数 (人)', 
                                   '求人広告合計', '職種数', '最小月収', '最小時給', '最大時給', 
                                   '販売', '最小月給(販売)', '最小時給(販売)', '最大時給(販売)',
                                   '飲食・フード', '最小月給(飲食・フード)', '最小時給(飲食・フード)', '最大時給(飲食・フード)',
                                   'オフィスワーク・事務', '最小月給(オフィスワーク・事務)',
                                   '営業', '最小月給(営業)', '最小時給(営業)', '最大時給(営業)',
                                   '講師・インストラクター', '最小時給(講師・インストラクター)', '最大時給(講師・インストラクター)',
                                   'ドライバー', '最小月給(ドライバー)', 
                                   '美容・エステ・ネイル', '最小月給(美容・エステ・ネイル)', '最小時給(美容・エステ・ネイル)', '最大時給(美容・エステ・ネイル)',
                                   '医療・看護・介護', '最小月給(医療・看護・介護)', '最小時給(医療・看護・介護)', '最大時給(医療・看護・介護)',
                                   '接客・サービス', '最小月給(接客・サービス)', '最小時給(接客・サービス)', '最大時給(接客・サービス)',
                                   'アミューズメント', '最小月給(アミューズメント)', '最小時給(アミューズメント)', '最大時給(アミューズメント)',
                                   '軽作業・清掃・警備・引越し', '最小月給(軽作業・清掃・警備・引越し)', '最小時給(軽作業・清掃・警備・引越し)', '最大時給(軽作業・清掃・警備・引越し)',
                                   '製造・組立・建設・他作業', '最小月給(製造・組立・建設・他作業)', '最小時給(製造・組立・建設・他作業)', '最大時給(製造・組立・建設・他作業)',
                                   'ファッション・アパレル', '最小月給(ファッション・アパレル)', '最小時給(ファッション・アパレル)'])

In [43]:
geojson_list = DataFrame(
{'index': range(0, 47, 1),
'prefecture': ['北海道',
'青森県',
'岩手県',
'宮城県',
'秋田県',
'山形県',
'福島県',
'茨城県',
'栃木県',
'群馬県',
'埼玉県',
'千葉県',
'東京都',
'神奈川県',
'新潟県',
'富山県',
'石川県',
'福井県',
'山梨県',
'長野県',
'岐阜県',
'静岡県',
'愛知県',
'三重県',
'滋賀県',
'京都府',
'大阪府',
'兵庫県',
'奈良県',
'和歌山県',
'鳥取県',
'島根県',
'岡山県',
'広島県',
'山口県',
'徳島県',
'香川県',
'愛媛県',
'高知県',
'福岡県',
'佐賀県',
'長崎県',
'熊本県',
'大分県',
'宮崎県',
'鹿児島県',
'沖縄県']})

In [16]:
func = 'jobsmesh'
apikey = 'ANwbR7yIsQMZ2LM'

In [22]:
def makeData(address):
    latlng = callGeocode(address)
    lat = latlng[0]; lng = latlng[1]
    url = 'https://www.meshstats.xyz/meshstats/api.php?latitude='+str(lat)+'&longitude='+str(lng)+'&distance=10&func='+func+'&key='+apikey
    df = DataFrame(pd.read_csv(url))
    df["世界メッシュコード"] = df["世界メッシュコード"].astype('str').str[2:11]
    return df

In [48]:
def staticsVisualize(address, statics):
    latlng = callGeocode(address)
    lat = latlng[0]; lng = latlng[1]
    geojson = r'data/geojson/prefecture_12.json'
    meshmap = folium.Map(location=[lat, lng], zoom_start=12)
    meshmap.choropleth(geo_path=geojson, data=makeData(address),
             columns=['世界メッシュコード', statics],
             key_on='properties.code',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name=statics,
                      reset=True)
    return meshmap

In [ ]:
staticsVisualize('東京都港区芝公園４丁目２−８', '飲食・フード')